# About the GeoTIFF (.tif) Raster FIle Format'

## What is GeoTIFF?

A GeoTIFF is a standard .tif or image file format that includes spatial (georeferencing) info embedded in the .tif file as tags. These embedded tags are called tif tags. These tags can include the following metadata:
1. Spatial Extent: What area does this dataset cover?
2. CRS: What spatial projection/CRS is used to store the data?
3. Resolution: The data appears to be in raster format: This means it is composed of pixels. What area on the ground does each pixel cover.
4. nodata value
5. How manz lazers are in the .tif file

### Geotiffs in python

The rasterio package in Python allows us to both open geotiff files and also to directly access .tif tags. You can quickly view the spatial extent, CRS, and resolution of your raster data.

NOTE: not all GeoTIFFs contain tif tags!

In [2]:
import os
import rasterio as rio
import earthpy as et

# Get data and set wkdir
et.data.get_data("colorado-flood")
os.chdir(os.path.join(et.io.HOME, "earth-analytics"))

Extracted output to C:\Users\Chris\earth-analytics\data\colorado-flood\.


In [7]:
# Define relative path to file
lidar_dem_path = os.path.join("data", "colorado-flood", "spatial",
                             "boulder-leehill-rd", "pre-flood", "lidar",
                             "pre_DTM.tif")

with rio.open(lidar_dem_path) as lidar_dem:
    lidar_dem.bounds

You can view spatial attributes associated with the raster file too. Below zou explore viewing a general list of attributes and then specific attributes including number of bands and horizontal (x, y) resolution.

In [8]:
# View generate metadata associated with the raster file
lidar_dem.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -3.4028234663852886e+38,
 'width': 4000,
 'height': 2000,
 'count': 1,
 'crs': CRS.from_epsg(32613),
 'transform': Affine(1.0, 0.0, 472000.0,
        0.0, -1.0, 4436000.0)}

In [10]:
# What is the spatial resolution?
lidar_dem.res

(1.0, 1.0)

You can access the tif tags as well.

In [12]:
# View image structure
with rio.open(lidar_dem_path) as lidar_dem:
    print(lidar_dem.tags(ns=("IMAGE_STRUCTURE")))
    lidar_dem_mask = lidar_dem.dataset_mask()

{'COMPRESSION': 'LZW', 'INTERLEAVE': 'BAND'}


### Raster Masks

You can view the mask associated with the data too. Here values that =0 are nodata values whereas = 255 are usable data values.

In [14]:
# View data mask
lidar_dem_mask

array([[  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       ...,
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255]], dtype=uint8)

* Data Tip: Read more about attributes associated with rasterio objects and how they map to gdal objects.

The info returned from the various attributes called above includes:
- x and y resolution
- projection
- data format (in this case your data are stored in float format which means they contain decimals)

and more.

You can also extract or view individual metadata attributes.

In [15]:
print(lidar_dem.crs)
print(lidar_dem.bounds)

EPSG:32613
BoundingBox(left=472000.0, bottom=4434000.0, right=476000.0, top=4436000.0)


In [18]:
# Define relative path to file
lidar_dsm_path = os.path.join("data", "colorado-flood", "spatial",
                             "boulder-leehill-rd", "pre-flood", "lidar",
                             "pre_DSM.tif")

# Open lidar dsm
with rio.open(lidar_dsm_path) as lidar_dsm:
    extent_lidar_dsm = lidar_dsm.bounds
    
if lidar_dem.bounds == lidar_dsm.bounds:
    print("Both datasets cover the same spatial extent")

Both datasets cover the same spatial extent


In [20]:
# Do both layers have the same spatial resolution?
lidar_dsm.res == lidar_dem.res

True

### Extra Metadata with EPSG

EPSG is short-hand way of specifying many CRS parameters at once. Each EPSG code correspondings to a Proj4code. In rasterio, more metadata is available if zou use Proj4 instead of EPSG.

In [21]:
# Get CRS of a rasterio object
lidar_dem.crs.data

{'init': 'epsg:32613'}

You can use et.epsg look up to get the proj4string for an epsg code too.

In [23]:
et.epsg["32613"]

'+proj=utm +zone=13 +datum=WGS84 +units=m +no_defs'

## Single layer (or Band) vs Multi-layer (Band Geotiffs)

GeoTIFFs can store more than 1 band or layer. You can see if a raster object has more than 1 layer using the .count() function

In [24]:
print(lidar_dem.count)

1


In [25]:
# How many bands/layers does the object have?
print("number of bands", lidar_dem.indexes)

number of bands (1,)
